In [ ]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

In [ ]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [ ]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input

In [ ]:
read_cc = True and False
read_tb = True and False

In [ ]:
cc = CasosConfirmados()
if read_cc:
    cc.read_excel()
    cc.save()
else:
    cc.load()

In [ ]:
casosc = cc.get_casos()
casosc.loc[casosc['obito']!='SIM','obito'] = 'NAO'
print(f"casosc len: {len(casosc)}\n")
casosc.groupby('obito')[['sexo']].count()

In [ ]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
casost.groupby('obito')[['sexo']].count()

In [ ]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

In [ ]:
del casosc['identificacao']

In [ ]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

In [ ]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

In [ ]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag'].values==ordem_duplicado['hash_diag'].values)

In [ ]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')
id_duplicado.loc[~id_duplicado['hash_diag'].isin(ordem_duplicado['hash_diag'])]

In [ ]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

In [ ]:
try:
    casosct = casosct_bkp.copy()
except:
    casosct_bkp = casosct.copy()
    
# casosct = casosct.sort_values(['comunicacao','nome','idade']).copy()
casosct = casosct.sort_values(['ordem']).copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
casosc = cc.get_casos()
casosct.loc[~casosct['ordem'].isin(casosc['ordem'])]

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)

In [ ]:
casosct['uf_resid'] = 'PR'
casosct.loc[casosct['ibge7']==9999999,'uf_resid'] = casosct.loc[casosct['ibge7']==9999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge7']==9999999,'mun_resid'] = casosct.loc[casosct['ibge7']==9999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge7']==9999999].drop(columns=['ibge7']),municipios[['ibge7','mun_resid','uf']].rename(columns={'uf':'uf_resid'}), on=['mun_resid','uf_resid'], how='left')
# casos_fora.loc[casos_fora['ibge7'].isna(),'excluir'] = 'SIM'
casos_fora.loc[casos_fora['ibge7'].isna(),'ibge7'] = 9999999

In [ ]:
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge7'] = casosct['ibge7'].apply(int)
casosct['idade'] = casosct['idade'].astype(int)

casosct.groupby(['evolucao'])[['sexo']].count()

In [ ]:
#####
## novos casos confirmados
#

casosct = casosct.sort_values(['comunicacao','nome'])
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge7'].apply(lambda x: int(str(x)[:6]))
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(lambda x: int(str(x)[:6]))

casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna(), 'hash_obito'] = casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna()].apply(
        lambda row: normalize_hash(row['nome']) + date_hash(row['data_com_evolucao']), axis=1
)

In [ ]:
casosct['hash'] = (casosct['nome'].apply(normalize_hash) +
                  casosct['idade'].astype(str) +
                  casosct['ibge_resid'].astype(str)  )

casosct['hash_less'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x - 1)) +
                        casosct['ibge_resid'].astype(str)  )

casosct['hash_more'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x + 1)) +
                        casosct['ibge_resid'].astype(str)  )

casosct['hash_diag'] = (casosct['nome'].apply(normalize_hash) + casosct['dt_diag'].apply(date_hash))

casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]#,'hash_atend','hash_less_atend','hash_more_atend','hash_diag','hash_obito']]
# casosct = casosct.set_index('identificacao')
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
novos_casos = pd.read_excel(join(default_input,'novos_casos.xlsx')).rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'nome_exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
novos_obitos = pd.read_excel(join(default_input,'novos_obitos.xlsx')).rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today() - timedelta(days=1))
novos_obitos['evolucao'] = 2

In [ ]:
novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
casosct.head()

In [ ]:
casosct.tail()

In [ ]:
casosct.to_pickle('casosct_21_05_2021.pkl')

In [ ]:
macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

In [ ]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

In [ ]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

In [ ]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

In [ ]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

In [ ]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casosct.columns

In [ ]:
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [ ]:
exit()

In [ ]:
writer = pd.ExcelWriter("casos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in casos_confirmados['macro'].unique():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==macro].copy()
    casos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,casos_macro,macros_idx_labels[macro])


writer.save()

In [ ]:
writer = pd.ExcelWriter("obitos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in obitos_confirmados['macro'].unique():
    obitos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==macro].copy()
    obitos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,obitos_macro,macros_idx_labels[macro])


writer.save()

In [ ]:
writer = pd.ExcelWriter("casos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in casos_confirmados['regional'].unique():
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [ ]:
writer = pd.ExcelWriter("obitos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in obitos_confirmados['regional'].unique():
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()